In [42]:
import pandas as pd

In [43]:
# Path to Data
PATHS = {'data': '../data/rows.csv'}

In [44]:
df = pd.read_csv(PATHS['data'], low_memory=False)

In [45]:
print(f'CFPB Data Shape: ({df.shape[0]:,}, {df.shape[1]})')
display(df.head(2))

CFPB Data Shape: (1,282,355, 18)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,05/10/2019,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,NaN,NaN,NAVY FEDERAL CREDIT UNION,FL,328XX,Older American,NaN,Web,05/10/2019,In progress,Yes,NaN,3238275
1,05/10/2019,Checking or savings account,Other banking product or service,Managing an account,Deposits and withdrawals,NaN,NaN,BOEING EMPLOYEES CREDIT UNION,WA,98204,NaN,NaN,Referral,05/10/2019,Closed with explanation,Yes,NaN,3238228


In [46]:
# Uniques for each field
tmp = df.describe(include='all').loc['unique', :]
display(tmp.to_frame().style.format("{:,.0f}"))
del tmp


,unique
Date received,"2,717"
Product,18
Sub-product,76
Issue,167
Sub-issue,218
Consumer complaint narrative,"366,945"
Company public response,10
Company,"5,275"
State,63
ZIP code,"22,591"


In [47]:
# Number of nulls in Product
print(f'# of Nulls in Product: {df.Product.isnull().sum()}')

# Counts for each value in field 'Product'
tmp = df.Product.value_counts().to_frame().style.format("{:,.0f}")
display(tmp)
del tmp

# of Nulls in Product: 0


,count
Product,
Mortgage,"278,098"
Debt collection,"244,873"
"Credit reporting, credit repair services, or other personal consumer reports","225,978"
Credit reporting,"140,432"
Credit card,"89,190"
Bank account or service,"86,206"
Student loan,"51,685"
Credit card or prepaid card,"47,653"
Checking or savings account,"40,641"


In [52]:
# Number of nulls in source fields
source_fields = ['Consumer complaint narrative',
                 'State',
                 'Company response to consumer']
print(f'Number of Non-Nulls in Each Column BEFORE Removing Nulls')
n_rows = len(df)
for col in source_fields:
    print(f'\t{col}: {n_rows - df[col].isnull().sum():,}')

# Reduce the dataframe to only non-null consumer complaints
data = (df.dropna(subset=source_fields)
        .reset_index(drop=True))
print((f'CFPB Data Shape After Removing Nulls:'
       f'({data.shape[0]:,}, {data.shape[1]})'))

# Number of nulls in source fields AFTER removing Nulls
print(f'Number of Non-Nulls in Each Column AFTER Removing Nulls')
n_rows = len(data)
for col in source_fields:
    print(f'\t{col}: {n_rows - data[col].isnull().sum():,}')

Number of Non-Nulls in Each Column BEFORE Removing Nulls
	Consumer complaint narrative: 383,564
	State: 1,262,955
	Company response to consumer: 1,282,348
CFPB Data Shape After Removing Nulls:(382,174, 18)
Number of Non-Nulls in Each Column AFTER Removing Nulls
	Consumer complaint narrative: 382,174
	State: 382,174
	Company response to consumer: 382,174
